### simple simulation - no transaction fee, no etf expenses, no taxes, no dividends

# 초기 조건 / 데이터 임포트 / 벤치마크 계산

## 라이브러리 임포트 / 초기 조건 설정

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [4]:
transaction_fee = 0.0025
etf_expenses = 0.0009
tax_rate = 0.22
initial_cash = 10000

In [5]:
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [6]:
strategies = []
returns = []

## 데이터 임포트

In [7]:
df = pd.read_csv('/content/drive/MyDrive/quant_projects/momentum_strategies/SPY.csv')[:-2] # 2021-09까지 결과 출력
closes = df.Close.to_numpy()
opens = df.Open.to_numpy()

In [8]:
tb3_df = pd.read_csv('/content/drive/MyDrive/quant_projects/momentum_strategies/TB3MS.csv')
risk_free_rates = tb3_df.TB3MS.to_numpy() / 100

## 벤치마크 계산

In [9]:
benchmark_shares = initial_cash // closes[0]
benchmark = closes * benchmark_shares

In [10]:
# 출처: http://blog.quantylab.com/mdd.html
def get_mdd(x):
    """
    MDD(Maximum Draw-Down)
    :return: (peak_upper, peak_lower, mdd rate)
    """
    arr_v = np.array(x)
    peak_lower = np.argmax(np.maximum.accumulate(arr_v) - arr_v)
    peak_upper = np.argmax(arr_v[:peak_lower])
    return peak_upper, peak_lower, (arr_v[peak_lower] - arr_v[peak_upper]) / arr_v[peak_upper]

In [11]:
# highs = df.High.to_numpy()
# lows = df.Low.to_numpy()

In [12]:
# np.argmax(np.maximum.accumulate(highs))

In [13]:
# 월중 mdd 
# max((np.maximum.accumulate(highs) - lows) / np.maximum.accumulate(highs))

In [14]:
# 월중 mdd
# peak_lower = np.argmax((np.maximum.accumulate(highs) - lows) / np.maximum.accumulate(highs))
# peak_upper = np.argmax(highs[:peak_lower])
# print(peak_upper, peak_lower, (lows[peak_lower] - highs[peak_upper]) / highs[peak_upper])

In [15]:
# get_mdd(benchmark)

In [16]:
def annualize(closes, opens, month_lag):
  annual_returns = []
  for i in range(len(closes) // 12 +1):
    if (i+1)*12 > len(closes):
      annual_returns.append(round((closes[-1] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))
    elif i*12-month_lag < 0:
      annual_returns.append(round((closes[i*12+11-month_lag] - opens[0]) / opens[0], 4))
    else:
      annual_returns.append(round((closes[i*12+11-month_lag] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))
  return annual_returns

In [17]:
def statistics(name, closes, annual_returns, current_flag=False):
  mean_risk_free_rate = np.mean(risk_free_rates) if current_flag else 0.005
  cagr = round(pow(closes[-1] / closes[0], 1/ (round(len(closes) / 12))) - 1, 4)
  mean_annual = round(np.mean(annual_returns),4)
  stddev = round(np.std(annual_returns),4)
  sharpe = round((np.mean(annual_returns)- mean_risk_free_rate) / np.std(annual_returns),4)
  worst_year = round(min(annual_returns), 4)
  best_year = round(max(annual_returns), 4)
  mdd = round(get_mdd(closes)[2], 4)
  return {'name':name, 'initial_balance': round(closes[0], 2), 'final_balance':round(closes[-1],2), 'cagr':cagr, 'mean_annual':mean_annual, 'stddev':stddev, 'sharpe':sharpe, 'worst_year':worst_year, 'best_year': best_year, 'mdd': mdd}

In [18]:
strategies.append(statistics('Benchmark', benchmark, annualize(closes, opens, 1)))
returns.append({"name":'Benchmark', "df":benchmark})

In [19]:
strategies

[{'best_year': 0.3453,
  'cagr': 0.0814,
  'final_balance': 96556.5,
  'initial_balance': 9991.41,
  'mdd': -0.522,
  'mean_annual': 0.0922,
  'name': 'Benchmark',
  'sharpe': 0.5243,
  'stddev': 0.1664,
  'worst_year': -0.3842}]

# Simulation

In [20]:
start_lag = 2 # data starts from 1993-02
rebalancing_periods = [1,2,3,4,6,12]
look_backs = [1,2,3,4,6,12]
for rebalancing_period in rebalancing_periods:
  for look_back in look_backs:
    for index in range(rebalancing_period):
      # 2개월 리밸런싱 -> 홀수달/짝수달, 3개월 리밸런싱 -> 0/1/2, 4개월 리밸런싱 -> 0/1/2/3
      for exclude_period in range(min(3, look_back)):
        df_temp = pd.DataFrame({'Date': df['Date'], 'Close': df['Close'], 'Open': df['Open']})
        simulation_detail = f'{index + start_lag}to{index + start_lag + rebalancing_period}ex{exclude_period}'
        name = f'SPY_simulation_{rebalancing_period}:{look_back}_{simulation_detail}'
        cash, month_closes, month_opens = [np.array([initial_cash] * len(closes), dtype=np.float64) for _ in range(3)]
        upwards, positions, shares = [np.zeros(len(closes)) for _ in range(3)]
        for i in range(12, len(closes)):
          # 12개월까지는 모든 포트폴리오가 현금보유, 벤치마크는 0개월부터 투자시작
          #조건에 만족하면 배당수익 기록 (보유 현금 증가로)
          # cash[i] = cash[i - rebalancing_period] dividends[i] * shares[i - rebalancing_period]
          if i % rebalancing_period == index:
            upwards[i] = 1 if (closes[i-exclude_period] - closes[i-look_back]) / closes[i-look_back] >= risk_free_rates[i] else 0
            shares[i] = (shares[i - 1] * closes[i] + cash[i - 1]) * upwards[i] // closes[i]
            shares_delta = shares[i] - shares[i - 1]
            month_opens[i] = cash[i-1] + shares[i-1] * opens[i]
            if shares_delta == 0:
              # 보유
              cash[i] = cash[i - 1] #만약 지난달에 배당을 받았다면 주식:현금 비율이 변화하므로 리밸런싱 필요
              positions[i] = shares[i] * closes[i]
            else:
              # 추가 매수, 매도 공통
              cash[i] = cash[i - 1] - (shares_delta) * closes[i] #shares_delta가 양수면 매수, 음수면 매도
              positions[i] = closes[i] * shares[i]
              # 거래비용 기록
              if shares_delta < 0:
                # 매도로 인한 손익 기록
                pass
          else:
            month_opens[i] = cash[i-1] + shares[i-1] * opens[i]
            cash[i] = cash[i-1]
            shares[i] = shares[i-1]
            positions[i] = shares[i] * closes[i]
          month_closes[i] = cash[i] + positions[i]
        df_temp['Upward'] = upwards.tolist()
        df_temp['Shares'] = shares.tolist()
        df_temp['Position'] = positions.tolist()
        df_temp['Cash'] = cash.tolist()
        df_temp['Month_close'] = month_closes.tolist()
        df_temp['Month_open'] = month_opens.tolist()
        df_temp['Benchmark'] = benchmark.tolist()
        # df_temp.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_{name}.csv')
        strategies.append(statistics(name, month_closes, annualize(month_closes, month_opens, 1)))
        returns.append({"name":name, "df":df_temp['Month_close'].to_numpy()})

In [21]:
strategies

[{'best_year': 0.3453,
  'cagr': 0.0814,
  'final_balance': 96556.5,
  'initial_balance': 9991.41,
  'mdd': -0.522,
  'mean_annual': 0.0922,
  'name': 'Benchmark',
  'sharpe': 0.5243,
  'stddev': 0.1664,
  'worst_year': -0.3842},
 {'best_year': 0.2664,
  'cagr': 0.0351,
  'final_balance': 27171.36,
  'initial_balance': 10000.0,
  'mdd': -0.2023,
  'mean_annual': 0.0358,
  'name': 'SPY_simulation_1:1_2to3ex0',
  'sharpe': 0.344,
  'stddev': 0.0895,
  'worst_year': -0.1429},
 {'best_year': 0.273,
  'cagr': 0.0421,
  'final_balance': 33079.01,
  'initial_balance': 10000.0,
  'mdd': -0.1165,
  'mean_annual': 0.0429,
  'name': 'SPY_simulation_1:2_2to3ex0',
  'sharpe': 0.422,
  'stddev': 0.0899,
  'worst_year': -0.0913},
 {'best_year': 0.2519,
  'cagr': 0.0284,
  'final_balance': 22496.02,
  'initial_balance': 10000.0,
  'mdd': -0.3392,
  'mean_annual': 0.0301,
  'name': 'SPY_simulation_1:2_2to3ex1',
  'sharpe': 0.2758,
  'stddev': 0.0912,
  'worst_year': -0.2372},
 {'best_year': 0.2736,
  '

# 결과 출력

## csv / jpg 출력

In [22]:
statistics_df = pd.DataFrame(strategies)
print(statistics_df)
print(statistics_df.iloc[statistics_df['final_balance'].idxmax()])

                               name  initial_balance  ...  best_year     mdd
0                         Benchmark          9991.41  ...     0.3453 -0.5220
1        SPY_simulation_1:1_2to3ex0         10000.00  ...     0.2664 -0.2023
2        SPY_simulation_1:2_2to3ex0         10000.00  ...     0.2730 -0.1165
3        SPY_simulation_1:2_2to3ex1         10000.00  ...     0.2519 -0.3392
4        SPY_simulation_1:3_2to3ex0         10000.00  ...     0.2736 -0.1795
..                              ...              ...  ...        ...     ...
416  SPY_simulation_12:12_12to24ex1         10000.00  ...     0.3084 -0.4590
417  SPY_simulation_12:12_12to24ex2         10000.00  ...     0.3040 -0.4588
418  SPY_simulation_12:12_13to25ex0         10000.00  ...     0.3050 -0.2550
419  SPY_simulation_12:12_13to25ex1         10000.00  ...     0.3050 -0.2559
420  SPY_simulation_12:12_13to25ex2         10000.00  ...     0.3050 -0.4841

[421 rows x 10 columns]
name               SPY_simulation_6:12_4to10ex2
ini

In [23]:
statistics_df.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_statistics.csv')

In [24]:
returns_df = pd.DataFrame({'Date': df['Date']})
for portfolio in returns:
  returns_df[portfolio['name']] = portfolio['df']
print(returns_df)
returns_df.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_returns.csv')

           Date  ...  SPY_simulation_12:12_13to25ex2
0    1993-02-01  ...                    10000.000000
1    1993-03-01  ...                    10000.000000
2    1993-04-01  ...                    10000.000000
3    1993-05-01  ...                    10000.000000
4    1993-06-01  ...                    10000.000000
..          ...  ...                             ...
339  2021-05-01  ...                    45125.196506
340  2021-06-01  ...                    45983.335329
341  2021-07-01  ...                    47101.486613
342  2021-08-01  ...                    48497.835329
343  2021-09-01  ...                    46098.897148

[344 rows x 422 columns]


In [ ]:
chart = returns_df.plot.line(x='Date', grid=True, figsize=(40,25), xticks=np.arange(0, 346, 12), yticks=np.arange(0, 150000, 10000))
chart.tick_params(axis='x', rotation=90)
chart.figure.savefig(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_returns.jpg')